In [10]:
import numpy as np
import pandas as pd
import math
import os

In [2]:
from datetime import datetime
todays_date = datetime.today().strftime('%m/%d/%Y')

In [3]:
pilot_or_exp = 1 # 1 pilot, 2 experimental

In [7]:
# Desired iterations:

# Pilot_testing, pilot_sona1 (1), pilot_prolific (10)
# Pilot_prolific_inform_memory (10), pilot_prolific_repeat_encoding (10)
# Pilot_prolific_no_filler (10), pilot_prolific_kitchen_sink (10), pilot_prolific_20_trials (8), pilot_prolific_afc (8), pilot_prolific_test_typicals (8)
# Pilot_prolific_afc2 (8), pilot_prolific_afc2_color (8), prolific_afc_50_subjects (50)
# Pilot_prolific_afc_0_25 (8)
# Pilot_sona_afc2
# –changed collection to main_task_2 – Pilot_sona_afc2 continues
# Sona_afc3, prolific_afc3: Flipped AFC pairing strategy
# –changed collection to main_task_3 – (accidentally did “sona_af3”)
# Main_task_4: prolific_afc4 (0-25)
# Maint_task_5: all retrieval 00C, prolific_afc5

In [107]:
template_path = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/demographics_template.csv'
datapath = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/data/main/afc_across_unused/'
dempath = '/Users/chrisiyer/_Current/thesis_folder/face_experiment/results/data/main/demographics/'
outputfile = dempath + 'submitted to redcap/main_for_redcap_3.csv'

In [108]:
task_files = [i for i in os.listdir(datapath) if i.startswith('taskdata')]
df_list = []
for f in task_files:
    df_list.append(pd.read_csv(datapath + f))
df = pd.concat(df_list)
df_dem = df[df.trial_type == 'demographics']

In [109]:
pro_dem_files = [i for i in os.listdir(dempath) if i.endswith('csv')]
df_list = []
for f in pro_dem_files:
    df_list.append(pd.read_csv(dempath + f))
df_pro = pd.concat(df_list)

In [110]:
output = pd.read_csv(template_path)

In [111]:
def create_row(record_id, race, hispanic, gender, other_gender, age, date):
    # race: ['1', '2', '3', '4', '5', '6', '99'] or '' at each
    # hispanic: 0 for no, 1 for yes
    # gender: 0 for male, 1 for female, 2 for other
    # other gender: text
    # age is just a number
    # date: of experimental completion
    
    row = {
        "record_id": record_id,
        "dateofentry": todays_date,
        "experimenter_name": 1, # tyler bonnen
        "irb_num": 2, # 10944
        "grant_sub": 99, # other
        "other_grant": "Anderson grant to tyler bonnen",

        "sub_gender": gender, #0 male 1 female 2 other
        "other_gender": other_gender,
        "sub_race___1": race[0], #native
        "sub_race___2": race[1], #asian
        "sub_race___3": race[2], #pacific islander
        "sub_race___4": race[3], #black
        "sub_race___5": race[4], #white
        "sub_race___6": race[5], #latinx
        "sub_race___99": race[6], #other
        "sub_hispanic": hispanic, #0 no 1 yes
        "sub_age": age, 
        "data_collection_date": date,
        "data_type": pilot_or_exp,
        "modality___1": 1, # behavioral
        "modality___2": "",
        "modality___3": "",
        "modality___4": "",
        "modality___5": "",
        "modality___6": "",
        "modality___7": "",
        "modality___8": "",
        "modality_other": "",
        "collected_online": 1, #yes
        "online_platform___1": 0, #mturk
        "online_platform___2": 1, #prolific,
        "online_platform___3": 0,
        "online_platform___4": 0,
        "other_online": "",
        "participant_exclusion": 0, #no
        "why_excluded": "",
        "participant_withdraw": 0, #no
        "why_withdraw": "",
        "comments": "",
        "date_renewal": "",
        "renewal_report": "",
        "demographics_complete": 0, #incomplete, I guess?
        "experimenter": "",
        "protocol": "",
        "date": "",
        "description": "",
        "report_required": "",
        "why_no_report": "",
        "date_form_submitted": "",
        "date_form_approved": "",
        "date_reported_renewal": "",
        "adverse_events_complete": ""
    }
    return row

In [112]:
def get_race(race_raw):
    output = [0]*7
    if 'Native' in race_raw: output[0] = '1'
    if 'Asian' in race_raw: output[1] = '1'
    if 'Pacific Islander' in race_raw: output[2] = '1'
    if 'Black' in race_raw: output[3] = '1'
    if 'White' in race_raw: output[4] = '1'
    if 'Latinx' in race_raw: output[5] = '1'
    if 'Other' in race_raw: output[6] = '1'
    return output
    
def get_gender(gender_raw): 
    if (gender_raw.startswith('F') or gender_raw.startswith('f')):
        return 1
    elif (gender_raw.startswith('M') or gender_raw.startswith('m')):
        return 0
    else:
        return 2

In [113]:
for i in range(len(df_dem)):
    
    sub_id = df_dem.subject_id.iloc[0]
    pro_row = df_pro[df_pro.participant_id == sub_id]
    
    if pro_row.status.iloc[0] == 'APPROVED' or pro_row.status.iloc[0] == 'AWAITING REVIEW':
        # record this participant
        # first, get race, gender
        race_raw = df_dem.iloc[i]['race']
        hisp_text = df_dem.iloc[i]['latinx']
        if hisp_text == "No": hispanic = 0
        else: hispanic = 1
        gender_raw = df_dem.iloc[i]['gender']
        if gender_raw == 'other':
            gender_raw = df_dem.iloc[i]['gender_other']

        if not isinstance(gender_raw, str): need_gender = True
        else: need_gender = False

    #     # USE THIS FOR FULL FORM
    #     age = df_exp.iloc[i,:]['age']
    #     date_time = df_exp.iloc[i,:]['datetime']
    #     date = datetime.strptime(date_time[0:date_time.index(',')], '%m/%d/%Y').strftime('%m/%d/%Y')

        # USE THIS NORMALLY (just need age and date from df_pro, maybe gender)
        age = pro_row.age
        date = pro_row.completed_date_time.iloc[0].split(" ")[0]
        if need_gender: gender_raw = pro_row['Sex'].iloc[0]


        row = create_row(
            record_id = i,
            race=get_race(race_raw), 
            hispanic=hispanic, 
            gender=get_gender(gender_raw), 
            other_gender = "", 
            age = age, 
            date = date)
        output.loc[len(output)] = row
    
output

,record_id,dateofentry,experimenter_name,irb_num,grant_sub,other_grant,sub_gender,other_gender,sub_race___1,sub_race___2,...,protocol,date,description,report_required,why_no_report,date_form_submitted,date_form_approved,date_reported_renewal,adverse_events_complete,Unnamed: 52
0,0,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
1,1,09/12/2022,1,2,99,Anderson grant to tyler bonnen,0,,0,0,...,,,,,,,,,,NaN
2,2,09/12/2022,1,2,99,Anderson grant to tyler bonnen,0,,0,0,...,,,,,,,,,,NaN
3,3,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
4,4,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,1,...,,,,,,,,,,NaN
98,98,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
99,99,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN
100,100,09/12/2022,1,2,99,Anderson grant to tyler bonnen,1,,0,0,...,,,,,,,,,,NaN


In [114]:
output.iloc[:,0:output.shape[1]-1].to_csv(outputfile, index=False)